# **Building a Conversational Chatbot with Langchain**

# **Description:**
In this activity, let's walk through the process of using LangChain, an open-source framework that enables the development of applications with large language models (LLMs) like OpenAI’s GPT-3.5-turbo.


# **Steps to Perform:**

1. Set up the Environment
2. Define a Document Loader
3. Create a Document Splitter
4. Embed the Text and Save it in Vector Stores
5. Create a Retrieval Function
6. Run the Chatbot and Understand the Code


# **Step 1: Set up the Environment**


*   Import the necessary libraries.


In [1]:
__import__('pysqlite3')
import sys

sys.modules['sqlite3'] = sys.modules['pysqlite3']

In [2]:
#Import necessary libraries
import os
import openai
import sys

# **Step 2: Define a Document Loader**



*  Use a document loader like PyPDF to load information from a PDF file.








In [3]:
#Using PyPDF
from langchain.document_loaders import PyPDFLoader

Doc_loader = PyPDFLoader("bcg-2022-annual-sustainability-report-apr-2023.pdf")
extracted_text = Doc_loader.load()

# **Step 3: Create a Document Splitter**


*   Break down big pieces of text into smaller parts using text splitters.




In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter  = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)
splitted_text=text_splitter.split_documents(extracted_text)

# **Step 4: Embed the Text and Save it in Vector Stores**


*  Arrange a place to store and organize the text splits to make
 them searchable.
*  Employ OpenAIEmbeddings to create a pretrained model instance, saving the results in a specified directory path.




In [6]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [7]:
from langchain.vectorstores import Chroma

In [8]:
persist_directory = "chroma_vector_x"

In [9]:
vectordb = Chroma.from_documents(
    documents=splitted_text,
    embedding=embeddings,
    persist_directory=persist_directory
)

# **Step 5: Create a Retrieval Function**


*   Retrieve pertinent data from storage based on user input using a retriever.




In [10]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

/tmp/ipykernel_432/2902982056.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


In [11]:
from langchain.chains import RetrievalQA
Retriever_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       )

# **Step 6: Run the Chatbot and Understand the Code**


*   Set up the chatbot, run it and
interact with it.


In [ ]:
import time
while True:
        query = input("\nEnter a query: ")
        if query == "exit":
            break
        if query.strip() == "":
            continue

        # Get the answer from the chain
        start = time.time()

        res=Retriever_chain(query)


        # Print the result

        end = time.time()
        print("\n\n> Question:")

        print(query)

        print(f"\n> Answer (took {round(end - start, 2)} s.):")

        print(res['result'])


Enter a query:  what is this document is about?


/tmp/ipykernel_432/3129579495.py:12: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res=Retriever_chain(query)




> Question:
what is this document is about?

> Answer (took 3.63 s.):
This document is a report that covers various topics such as driving social impact, protecting our planet, and empowering our people. It provides information on the company's activities and initiatives in these areas. You can find more details in the sections "Driving Social Impact," "Protecting Our Planet," and "Empowering Our People" in the report.



Enter a query:  what is an interesting part of this document?




> Question:
what is an interesting part of this document?

> Answer (took 2.03 s.):
I don't have access to the specific document you are referring to. If you can provide me with some details or context from the document, I can help you identify an interesting part based on that information.


# **Conclusion:**
The code will ask the user to enter a query, get an answer from the chatbot, and print it along with how long it took to get the answer. The user can exit the chatbot by typing **exit**. If the user enters an empty query, the chatbot will ask for another query. This is the final step in creating and running the chatbot.